In [27]:
## Churn_modelling_customer_data
## Let's import the required libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
import pickle


In [28]:
## Let's read the data from the CSV file
data=pd.read_csv('churn_modelling.csv')
data.head(10)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
5,6,15574012,Chu,645,Spain,Male,44,8,113755.78,2,1,0,149756.71,1
6,7,15592531,Bartlett,822,France,Male,50,7,0.00,2,1,1,10062.80,0
7,8,15656148,Obinna,376,Germany,Female,29,4,115046.74,4,1,0,119346.88,1
8,9,15792365,He,501,France,Male,44,4,142051.07,2,0,1,74940.50,0
9,10,15592389,H?,684,France,Male,27,2,134603.88,1,1,1,71725.73,0


In [29]:
## now let's concentrate on removing the unwanted or the rows that won't effect the outcome(excited)
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


## Here the problem is Binary classification. So, like the output is 0 or 1.

1. for this purpose we need to take every feature in the numbers(numerical representation) for that we can use Labelencoder or one hot encoding

In [30]:
## let's do the label encoding for the gender colunmn

label_encoder_gender=LabelEncoder()
data['Gender']=label_encoder_gender.fit_transform(data['Gender'])

In [31]:

data.head(10)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
5,645,Spain,1,44,8,113755.78,2,1,0,149756.71,1
6,822,France,1,50,7,0.00,2,1,1,10062.80,0
7,376,Germany,0,29,4,115046.74,4,1,0,119346.88,1
8,501,France,1,44,4,142051.07,2,0,1,74940.50,0
9,684,France,1,27,2,134603.88,1,1,1,71725.73,0


## Here we cannot apply the label encoder for the geogrpahy(it has 'france','spain','germany')
1. reason is simple here we are training the model and predicting the ouput using numerical values so let's say if we apply the label encoder for the geography then the values are like it has('france'=0,'spain'=1,'germany'=2) machine thinks that the germany is greater than other 2 countries.

2. to avoid these we use one hot encoding which gives equal importance to all countries or features.

In [32]:
## let's apply one hot encoding to the Geography
onehot_encoding=OneHotEncoder()
geography_encoder=onehot_encoding.fit_transform(data[['Geography']])
geography_encoder.toarray()

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]], shape=(10000, 3))

In [33]:
## now let's see the one hot encoded columns and store them in a dataframe so that we can contact with original data frame
onehot_encoding.get_feature_names_out(['Geography'])
encoded_geo_df=pd.DataFrame(geography_encoder.toarray(),columns=onehot_encoding.get_feature_names_out(['Geography']))
encoded_geo_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [34]:
## let's concat this with original dataframe and drop the geography column
data=pd.concat([data.drop('Geography',axis=1),encoded_geo_df],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [35]:
## let's split the data and also apply the standard scaler to the data( if have a larger number for one feature it won't consider the smaller feature which in turns loses the accuracy)
x=data.drop(['Exited'],axis=1)
y=data['Exited']
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

# use standard scaler
scaler=StandardScaler()
x_train=scaler.fit_transform(x_train)
x_test=scaler.transform(x_test)


In [36]:
## save the Encoder's and Scaler
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('one_hot_encoding_geography.pkl','wb') as file:
    pickle.dump(onehot_encoding,file)

with open ('standard_scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

## ANN Implementation steps

1. Create a sequential network(which has input layer, hidden layers,output layer)
2. Dense (keras) it will specify how many neurons a hidden layer should have
3. Activation functions like : sigmoid, relu, leaky relu, softmax,tanh
4. optimizers :- this will update the weights in cinnections through backward propogation
5. loss functions :- to calculate the loss (Actual output-preidcted output)
6. metrics(Accuracy) for classification and for regression problems we use MSE(Mean Squared error) and Mean absolute error(MAE).
7. Training information-logs- will store in folder-- i will be able to use in tensor board--> to visualize the graphs.(Visualization)

In [37]:
## Import the required libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard


In [38]:
## now we need to create a model and in this model we will have a hidden layers. so, specify the no of neurons, activation function,input features.
model = Sequential([
    Dense(64,activation='relu',input_shape=(x_train.shape[1],)), ## we only need to specify the input feature number in first hidden layer 

    Dense(32,activation='relu'),## hidden layer 2
    Dense(1,activation='sigmoid')## output layer

])

C:\Users\nellu\AppData\Roaming\Python\Python313\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [39]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [40]:
import tensorflow
opt=tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss=tensorflow.keras.losses.BinaryCrossentropy()
loss

<LossFunctionWrapper(<function binary_crossentropy at 0x000002ABB8BE5940>, kwargs={'from_logits': False, 'label_smoothing': 0.0, 'axis': -1})>

In [41]:
## compile the model
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [42]:
# set up the tensor board
import datetime
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
log_dir='logs/fit'+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callbacks=TensorBoard(log_dir=log_dir,histogram_freq=1)

# Implementation of early stopping and importance

1. let's say my model is trianed better or giving better accuracy after 20 epoches but we are running 100 epoches then there is no point in running the training after that point.
2. Let's say after certain epoches the loss is not decreasing or may be like 1% or 2% then there is no point in runnning the epoches after that point. 
3. which is why we use earlystopping and a keyword patience which will watch a certain parameter values over the time and stopps the execution.
4. we monitor validation loss in this case and check if it is changing over the mentioned patience level or not.
5. Another parameter restore best weights is used to reload the best weights during forward or backward propagation while retrying or running again. it saves us time

In [43]:
## Setup the Early stopping
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [44]:
# Fit the data into the model and run 
history=model.fit(
    x_train,y_train,validation_data=(x_test,y_test),epochs=100,
    callbacks=[early_stopping_callback,tensorflow_callbacks]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.8140 - loss: 0.4406 - val_accuracy: 0.8440 - val_loss: 0.3906
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8439 - loss: 0.3822 - val_accuracy: 0.8550 - val_loss: 0.3569
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8551 - loss: 0.3558 - val_accuracy: 0.8585 - val_loss: 0.3509
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8594 - loss: 0.3452 - val_accuracy: 0.8595 - val_loss: 0.3513
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8599 - loss: 0.3403 - val_accuracy: 0.8610 - val_loss: 0.3430
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8614 - loss: 0.3365 - val_accuracy: 0.8575 - val_loss: 0.3431
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8620 - loss: 0.3333 - val_accuracy: 0.8585 - val_loss: 0.3438
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8646 - loss: 0.3297 - val_accu

In [45]:
## save the model

model.save("model_new.keras", save_format="keras_v3") # h5 files and .keras are compatible with keras

In [46]:
## launch the tensorboard.
%load_ext tensorboard



The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [47]:
%tensorboard --logdir logs/fit20251209-094109

Reusing TensorBoard on port 6006 (pid 28908), started 1 day, 1:08:46 ago. (Use '!kill 28908' to kill it.)

In [48]:
## Let's take an input and predict
input_data={
    'CreditScore':635,
    'Gender':'Male',
    'Age':25,
    'Tenure':3,
    'Balance':1000,
    'NumOfProducts':2,
    'HasCrCard'	:1,
    'IsActiveMember': 1,
    'EstimatedSalary' :50000,	
    'Geography':'Spain'
}

In [49]:
# let's use the model and open the pickle file and use them 
import pickle
from tensorflow.keras.models import load_model
model=load_model('model.h5')
with open('label_encoder_gender.pkl','rb') as file:
    encode_gender=pickle.load(file)
    
with open('one_hot_encoding_geography.pkl','rb') as file:
    encode_onehot=pickle.load(file)   

with open('standard_scaler.pkl','rb') as file:
    scaler=  pickle.load(file)

In [50]:
## for the given input data follow the same data preprocessing steps like before
## label encoding for gender
# convert dict type data to dataframe data type
input_df=pd.DataFrame([input_data])
input_df['Gender']=encode_gender.transform(input_df['Gender'])
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography
0,635,1,25,3,1000,2,1,1,50000,Spain


In [51]:
## perform one hot encoding for geography
encoded_geography=(encode_onehot.transform([input_df['Geography']]))
encoded_geography.toarray()
encoded_geography=pd.DataFrame(encoded_geography.toarray(),columns=encode_onehot.get_feature_names_out(['Geography']))
encoded_geography

C:\Users\nellu\AppData\Roaming\Python\Python313\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,0.0,0.0,1.0


In [52]:
# let's concat the entire data together
final_input_df=pd.concat((input_df.drop(['Geography'],axis=1),encoded_geography),axis=1)
scaled_input=scaler.transform(final_input_df)
prediction=model.predict(scaled_input)
prediction_proba=prediction[0][0]
if prediction_proba < 0.5:
    print('customer is likely to churn')
else:
    print('customer is not likely to churn')    

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
customer is likely to churn
